In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\Barytes\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.NBS as NBS
import numpy as np
CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 42
MDs = create_MDs(config, seed=seed)

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 10, 'l': 800, 'cn': 0.48727005942368123, 'Fn': 2.4014286128198323, 'kn': 0.6659969709057025, 'omega_n': 1, 'Rn': 9.270270025833746}
{'s': 10, 'l': 800, 'cn': 0.3779972601681013, 'Fn': 0.6161672243363989, 'kn': 0.7330880728874676, 'omega_n': 1, 'Rn': 7.653352487012544}
{'s': 10, 'l': 800, 'cn': 0.3102922471479012, 'Fn': 2.4398197043239884, 'kn': 0.7162213204002108, 'omega_n': 1, 'Rn': 6.796707114575109}
{'s': 10, 'l': 800, 'cn': 0.3917022549267169, 'Fn': 1.1084844859190754, 'kn': 0.5623782158161189, 'omega_n': 1, 'Rn': 7.979027325089509}
{'s': 10, 'l': 800, 'cn': 0.6059264473611897, 'Fn': 0.7789877213040837, 'kn': 0.44607232426760907, 'omega_n': 1, 'Rn': 7.104438488824009}
{'lambda0': 50, 'D0': 4, 'theta': 5, 'o': 10, 'omega_0': 2}


In [3]:
lamb, p, Dmax = NBS.ADMM(esp, MDs)
print(lamb)
print(p)
print(Dmax)
r = NBS.negotiation(esp, MDs,lamb, p, Dmax)
print(r)

c:\Users\Barytes\Desktop\NBS_project\src\NBS.py:69: RuntimeWarning: invalid value encountered in double_scalars
  term_u = np.sum([md.cn*(pn**2)+(md.Fn)**md.kn-(md.Fn-pn)**md.kn for md, pn in zip(MDs, p_vec)])
d:\Program\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:51: RuntimeWarning: invalid value encountered in minimum
  return getattr(obj, method)(*args, **kwds)
d:\Program\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:51: RuntimeWarning: invalid value encountered in maximum
  return getattr(obj, method)(*args, **kwds)


: 